In [1]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.preprocessing.image import img_to_array, load_img

# Load models
harassment_model_path = "harass.h5"
harassment_model = tf.keras.models.load_model(harassment_model_path)

gender_model_path = "gender_classification_model.h5"
gender_model = tf.keras.models.load_model(gender_model_path)

expression_model_path = "emotion_model.h5"
expression_model = tf.keras.models.load_model(expression_model_path)

# Define emotion labels
emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

base_model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

def process_frame_for_harassment(frame):
    temp_img_path = "temp_frame.jpg"
    cv2.imwrite(temp_img_path, frame)
    img = load_img(temp_img_path, target_size=(224, 224))
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    features = base_model.predict(img_array)
    features = features.reshape(1, 7 * 7 * 512)  # This may need adjusting
    predictions = harassment_model.predict(features)
    return np.argmax(predictions[0])

def process_frame_for_gender(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)
    
    male_count = 0
    female_count = 0
    lone_woman_detected = False
    detected_emotions = []

    for (x, y, w, h) in faces:
        margin = int(max(w, h) * 0.3)
        x1, y1 = max(0, x - margin), max(0, y - margin)
        x2, y2 = min(frame.shape[1], x + w + margin), min(frame.shape[0], y + h + margin)
        
        roi = frame[y1:y2, x1:x2]
        roi_resized = cv2.resize(roi, (64, 64))
        roi_array = img_to_array(roi_resized) / 255.0
        roi_array = np.expand_dims(roi_array, axis=0)
        
        prediction = gender_model.predict(roi_array)
        result = "Male" if prediction[0][0] >= 0.5 else "Female"
        
        if result == "Male":
            male_count += 1
        else:
            female_count += 1
            if male_count == 0:
                lone_woman_detected = True
        
        # Draw rectangle around the face
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        
        # Gender label above the bounding box
        label = f"{result} ({prediction[0][0]:.2f})"
        label_position = (x1, y1 - 10) if y1 - 10 > 10 else (x1, y2 + 30)
        cv2.putText(frame, label, label_position, cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    
    return frame, male_count, female_count, lone_woman_detected, detected_emotions

def process_frame_for_emotion(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)
    detected_emotions = []

    for (x, y, w, h) in faces:
        margin = int(max(w, h) * 0.3)
        x1, y1 = max(0, x - margin), max(0, y - margin)
        x2, y2 = min(frame.shape[1], x + w + margin), min(frame.shape[0], y + h + margin)
        
        face_roi = gray[y1:y2, x1:x2]
        face_roi_resized = cv2.resize(face_roi, (48, 48))
        face_roi_normalized = face_roi_resized / 255.0
        face_roi_normalized = face_roi_normalized.astype(np.float32)
        face_roi_rgb = cv2.cvtColor(face_roi_normalized, cv2.COLOR_GRAY2RGB)
        face_roi_expanded = np.expand_dims(face_roi_rgb, axis=0)
        
        try:
            emotion_prediction = expression_model.predict(face_roi_expanded)
            emotion_label = emotion_labels[np.argmax(emotion_prediction)]
            detected_emotions.append(emotion_label)
            
            # Emotion label below the bounding box
            label_position = (x1, y2 + 30) if y2 + 30 < frame.shape[0] else (x1, y1 - 10)
            cv2.putText(frame, emotion_label, label_position, cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
        except Exception as e:
            print("Error during emotion prediction:", str(e))
    
    return frame, detected_emotions

def process_video_from_camera():
    cap = cv2.VideoCapture(0)  # Use camera index 0 for the default camera

    # Set the window to fullscreen
    cv2.namedWindow('Video Feed', cv2.WINDOW_NORMAL)
    cv2.setWindowProperty('Video Feed', cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("Failed to grab frame")
            break

        harassment_prediction = process_frame_for_harassment(frame)
        frame, male_count, female_count, lone_woman_detected, detected_emotions = process_frame_for_gender(frame)
        frame, detected_emotions = process_frame_for_emotion(frame)
        
        # Check for harassment based on detected emotions
        emotion_criteria = ['Disgust', 'Angry', 'Fear']
        harassment_detected = any(emotion in detected_emotions for emotion in emotion_criteria)
        
        # Updated Criteria for Harassment Detection
        if male_count > 0 and female_count > 0 and harassment_prediction == 1:
            if harassment_detected:
                cv2.putText(frame, "!!! HARASSMENT DETECTED !!!", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0, 255), 4)
        
        if lone_woman_detected and male_count == 0:
            cv2.putText(frame, "!!! LONE WOMAN DETECTED !!!", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 255), 4)
        
        if lone_woman_detected and male_count > 0:
            cv2.putText(frame, f"LONE WOMAN SURROUNDED BY {male_count} MEN", (50, 150), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 0, 0), 4)

        # Display the resulting frame
        cv2.imshow('Video Feed', frame)

        # Exit condition
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

# Run the function to process the camera feed
process_video_from_camera()


c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\optimizers\base_optimizer.py:33: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:1284: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvNamedWindow'
